# Import Libraries

In [1]:
from tqdm import tqdm
import pandas as pd
import nltk
from nltk import word_tokenize
import string

from utils import load_sessions, read_session
from main import generate_buffer
from events import generate_event_seq
from summary import stats

# Compute summary statistics

In [2]:
sessions = load_sessions()
# sessions = load_sessions()[:10]

file_name = []
text = []
sentence_metrics_list = []
api_metrics_list = []

err = []

for sess in tqdm(sessions):
    events = read_session(sess, verbose=0)
    try:
        text_buffer = generate_buffer(events)
    except:
        err.append(str(sess.split('/')[-1]) + " is throwing an error!")
        continue
    file_name.append(sess.split('/')[-1])
    text.append(text_buffer[-1])
    event_seq_dict = generate_event_seq(buffer=text_buffer,
                                        events=events)
    sentence_metrics, api_metrics = stats(event_seq_dict)
    sentence_metrics_list.append(sentence_metrics)
    api_metrics_list.append(api_metrics)
    
for e in err:
    print(e)
    
df = pd.DataFrame()

df["file_name"] = file_name
df["text"] = text

for col in sentence_metrics_list[0]:
    df[str(col)] = [x[col] for x in sentence_metrics_list]
    
for col in api_metrics_list[0]:
    df[str(col)] = [x[col] for x in api_metrics_list]

Successfully downloaded 1447 writing sessions in CoAuthor!


  1%|▌                                             | 19/1447 [00:19<30:27,  1.28s/it]/home/sri/Documents/UTS Research/SLAC-GRAPH/test/summary.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if ("gpt3-call" not in seq) and ("prompt" not in seq) and ("user" in seq):
/home/sri/Documents/UTS Research/SLAC-GRAPH/test/summary.py:19: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if ("gpt3-call" in seq) and ("user" not in seq):
 20%|████████▉                                    | 287/1447 [04:32<20:28,  1.06s/it]/home/sri/Documents/UTS Research/SLAC-GRAPH/test/events.py:171: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if "prompt" in temp_dict["sequence"][idx]:
/home/sri/Documents/UTS Research/SLAC-GRAPH/test/summary.py:17: FutureWarning: elementw

312e3263a9f24f3184364949a42a6dfc.jsonl is throwing an error!


# Ratios

In [3]:
def get_ratio(num1, num2):
    return float(num1 / num2)


def add(num1, num2):
    return num1 + num2

In [4]:
# GPT-3 : Total Sentences

df["GPT-3 : Total Sentences"] = list(map(get_ratio, 
    df["Number of sentences completely authored by GPT-3"], 
    df["Total number of sentences"]
))

df["GPT-3 : Total Sentences"]

0       0.000000
1       0.000000
2       0.000000
3       0.031250
4       0.000000
          ...   
1441    0.000000
1442    0.000000
1443    0.371429
1444    0.000000
1445    0.117647
Name: GPT-3 : Total Sentences, Length: 1446, dtype: float64

In [5]:
# User : Total Sentences

df["User : Total Sentences"] = list(map(get_ratio, 
    df["Number of sentences completely authored by the user"], 
    df["Total number of sentences"]
))

df["User : Total Sentences"]

0       0.400000
1       0.500000
2       0.818182
3       0.562500
4       0.648649
          ...   
1441    0.534483
1442    0.687500
1443    0.085714
1444    0.666667
1445    0.411765
Name: User : Total Sentences, Length: 1446, dtype: float64

In [6]:
# Amount of usage of GPT-3 (SD+SE/SA)

df["Amount of GTP-3 Usage"] = list(map(get_ratio, 
    pd.Series(list(map(add, df["Number of sentences authored by GPT-3 and user"], 
                       df["Number of sentences completely authored by GPT-3"]))), 
    df["Total number of sentences"]
))

df["Amount of GTP-3 Usage"]

0       0.333333
1       0.200000
2       0.136364
3       0.375000
4       0.270270
          ...   
1441    0.396552
1442    0.062500
1443    0.742857
1444    0.166667
1445    0.235294
Name: Amount of GTP-3 Usage, Length: 1446, dtype: float64

In [7]:
# Type Token Ratio

def get_ttr(text):
    sentence_tokens = word_tokenize(text)
    punctuations = list(string.punctuation)
    sentence_tokens_clean = [word for word in sentence_tokens if word not in punctuations]
    ttr = len(set(sentence_tokens_clean)) / len(sentence_tokens_clean)
    return ttr


df["Type Token Ratio"] = df["text"].apply(get_ttr)
df["Type Token Ratio"]

0       0.564706
1       0.613559
2       0.548476
3       0.469974
4       0.491135
          ...   
1441    0.362854
1442    0.629758
1443    0.457584
1444    0.538710
1445    0.483871
Name: Type Token Ratio, Length: 1446, dtype: float64

# Mean

In [8]:
import numpy as np

print("Sentence Metrics")
for col in sentence_metrics_list[0]:
    print("Mean of", col, ":", np.mean(df[col]))
    
print("\nAPI Metrics")
for col in api_metrics_list[0]:
    print("Mean of", col, ":", np.mean(df[col]))
    
print("\nRatios")
print("Mean of GPT-3 / Total Sentences : ", np.mean(df["GPT-3 : Total Sentences"]))
print("Mean of User / Total Sentences : ", np.mean(df["User : Total Sentences"]))
print("Mean of Type Token Ratio : ", np.mean(df["Type Token Ratio"]))
print("Mean of Amount of GTP-3 Usage : ", np.mean(df["Amount of GTP-3 Usage"]))

Sentence Metrics
Mean of Total number of sentences : 28.96265560165975
Mean of Number of sentences of initial prompt : 4.421161825726141
Mean of Number of sentences completely authored by the user : 16.24273858921162
Mean of Number of sentences completely authored by GPT-3 : 0.6853388658367912
Mean of Number of sentences authored by GPT-3 and user : 7.6134163208852

API Metrics
Mean of Total number of GPT-3 calls made : 12.531120331950207
Mean of Number of times GPT-3 suggestion is used : 8.857538035961273
Mean of Number of times user rejected GPT-3 suggestion : 3.673582295988935
Mean of Number of times GPT-3 suggestion is modified : 1.586445366528354
Mean of Number of times GPT-3 suggestion is used as is : 7.271092669432918

Ratios
Mean of GPT-3 / Total Sentences :  0.021900159513948934
Mean of User / Total Sentences :  0.5412613590031221
Mean of Type Token Ratio :  0.48183805678712555
Mean of Amount of GTP-3 Usage :  0.2848898237822877


# Median

In [9]:
import numpy as np

print("Sentence Metrics")
for col in sentence_metrics_list[0]:
    print("Median of", col, ":", np.median(df[col]))
    
print("\nAPI Metrics")
for col in api_metrics_list[0]:
    print("Median of", col, ":", np.median(df[col]))

print("\nRatios")
print("Median of GPT-3 / Total Sentences : ", np.median(df["GPT-3 : Total Sentences"]))
print("Median of User / Total Sentences : ", np.median(df["User : Total Sentences"]))
print("Median of Type Token Ratio : ", np.median(df["Type Token Ratio"]))
print("Median of Amount of GTP-3 Usage : ", np.median(df["Amount of GTP-3 Usage"]))

Sentence Metrics
Median of Total number of sentences : 27.0
Median of Number of sentences of initial prompt : 4.0
Median of Number of sentences completely authored by the user : 15.0
Median of Number of sentences completely authored by GPT-3 : 0.0
Median of Number of sentences authored by GPT-3 and user : 6.0

API Metrics
Median of Total number of GPT-3 calls made : 10.0
Median of Number of times GPT-3 suggestion is used : 7.0
Median of Number of times user rejected GPT-3 suggestion : 3.0
Median of Number of times GPT-3 suggestion is modified : 1.0
Median of Number of times GPT-3 suggestion is used as is : 5.0

Ratios
Median of GPT-3 / Total Sentences :  0.0
Median of User / Total Sentences :  0.5641025641025641
Median of Type Token Ratio :  0.4821182605273514
Median of Amount of GTP-3 Usage :  0.25


# Standard Deviation

In [10]:
import numpy as np

print("Sentence Metrics")
for col in sentence_metrics_list[0]:
    print("Standard Deviation of", col, ":", np.std(df[col]))
    
print("\nAPI Metrics")
for col in api_metrics_list[0]:
    print("Standard Deviation of", col, ":", np.std(df[col]))
    
print("\nRatios")
print("Standard Deviation of GPT-3 / Total Sentences : ", np.std(df["GPT-3 : Total Sentences"]))
print("Standard Deviation of User / Total Sentences : ", np.std(df["User : Total Sentences"]))
print("Standard Deviation of Type Token Ratio : ", np.std(df["Type Token Ratio"]))
print("Standard Deviation of Amount of GTP-3 Usage : ", np.std(df["Amount of GTP-3 Usage"]))

Sentence Metrics
Standard Deviation of Total number of sentences : 10.388909909258523
Standard Deviation of Number of sentences of initial prompt : 2.3909859029112486
Standard Deviation of Number of sentences completely authored by the user : 9.535179050568926
Standard Deviation of Number of sentences completely authored by GPT-3 : 1.8864423445191325
Standard Deviation of Number of sentences authored by GPT-3 and user : 5.953072577616293

API Metrics
Standard Deviation of Total number of GPT-3 calls made : 9.204158194377401
Standard Deviation of Number of times GPT-3 suggestion is used : 7.424057788661343
Standard Deviation of Number of times user rejected GPT-3 suggestion : 3.530339833311101
Standard Deviation of Number of times GPT-3 suggestion is modified : 1.796857239727531
Standard Deviation of Number of times GPT-3 suggestion is used as is : 7.233591709071116

Ratios
Standard Deviation of GPT-3 / Total Sentences :  0.05414721819670053
Standard Deviation of User / Total Sentences 

# Minimum

In [11]:
import numpy as np

print("Sentence Metrics")
for col in sentence_metrics_list[0]:
    print("Minimum of", col, ":", np.min(df[col]))
    
print("\nAPI Metrics")
for col in api_metrics_list[0]:
    print("Minimum of", col, ":", np.min(df[col]))
    
print("\nRatios")
print("Minimum of GPT-3 / Total Sentences : ", np.min(df["GPT-3 : Total Sentences"]))
print("Minimum of User / Total Sentences : ", np.min(df["User : Total Sentences"]))
print("Minimum of Type Token Ratio : ", np.min(df["Type Token Ratio"]))
print("Minimum of Amount of GTP-3 Usage : ", np.min(df["Amount of GTP-3 Usage"]))

Sentence Metrics
Minimum of Total number of sentences : 11
Minimum of Number of sentences of initial prompt : 0
Minimum of Number of sentences completely authored by the user : 0
Minimum of Number of sentences completely authored by GPT-3 : 0
Minimum of Number of sentences authored by GPT-3 and user : 0

API Metrics
Minimum of Total number of GPT-3 calls made : 0
Minimum of Number of times GPT-3 suggestion is used : 0
Minimum of Number of times user rejected GPT-3 suggestion : 0
Minimum of Number of times GPT-3 suggestion is modified : 0
Minimum of Number of times GPT-3 suggestion is used as is : 0

Ratios
Minimum of GPT-3 / Total Sentences :  0.0
Minimum of User / Total Sentences :  0.0
Minimum of Type Token Ratio :  0.28794178794178793
Minimum of Amount of GTP-3 Usage :  0.0


# Maximum

In [12]:
import numpy as np

print("Sentence Metrics")
for col in sentence_metrics_list[0]:
    print("Maximum of", col, ":", np.max(df[col]))
    
print("\nAPI Metrics")
for col in api_metrics_list[0]:
    print("Maximum of", col, ":", np.max(df[col]))
    
print("\nRatios")
print("Maximum of GPT-3 / Total Sentences : ", np.max(df["GPT-3 : Total Sentences"]))
print("Maximum of User / Total Sentences : ", np.max(df["User : Total Sentences"]))
print("Maximum of Type Token Ratio : ", np.max(df["Type Token Ratio"]))
print("Maximum of Amount of GTP-3 Usage : ", np.max(df["Amount of GTP-3 Usage"]))

Sentence Metrics
Maximum of Total number of sentences : 78
Maximum of Number of sentences of initial prompt : 9
Maximum of Number of sentences completely authored by the user : 64
Maximum of Number of sentences completely authored by GPT-3 : 22
Maximum of Number of sentences authored by GPT-3 and user : 42

API Metrics
Maximum of Total number of GPT-3 calls made : 65
Maximum of Number of times GPT-3 suggestion is used : 47
Maximum of Number of times user rejected GPT-3 suggestion : 24
Maximum of Number of times GPT-3 suggestion is modified : 10
Maximum of Number of times GPT-3 suggestion is used as is : 47

Ratios
Maximum of GPT-3 / Total Sentences :  0.6111111111111112
Maximum of User / Total Sentences :  0.9629629629629629
Maximum of Type Token Ratio :  0.6879432624113475
Maximum of Amount of GTP-3 Usage :  0.8666666666666667


# Correlation

In [13]:
df.corr()

/tmp/ipykernel_3364/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,Total number of sentences,Number of sentences of initial prompt,Number of sentences completely authored by the user,Number of sentences completely authored by GPT-3,Number of sentences authored by GPT-3 and user,Total number of GPT-3 calls made,Number of times GPT-3 suggestion is used,Number of times user rejected GPT-3 suggestion,Number of times GPT-3 suggestion is modified,Number of times GPT-3 suggestion is used as is,GPT-3 : Total Sentences,User : Total Sentences,Amount of GTP-3 Usage,Type Token Ratio
Total number of sentences,1.000000,-0.129523,0.773865,0.218429,0.488421,0.365100,0.454037,-0.002934,0.051890,0.453102,0.090754,0.265854,0.025049,-0.527044
Number of sentences of initial prompt,-0.129523,1.000000,-0.300117,-0.062613,-0.127129,-0.067734,-0.096746,0.026855,-0.056362,-0.085293,-0.060789,-0.377381,-0.116840,0.034719
Number of sentences completely authored by the user,0.773865,-0.300117,1.000000,-0.083143,-0.104341,-0.218031,-0.167396,-0.216420,-0.100297,-0.146890,-0.181537,0.779995,-0.521486,-0.291106
Number of sentences completely authored by GPT-3,0.218429,-0.062613,-0.083143,1.000000,0.222622,0.422258,0.451338,0.151763,0.199498,0.413665,0.945054,-0.281658,0.403351,-0.171809
Number of sentences authored by GPT-3 and user,0.488421,-0.127129,-0.104341,0.222622,1.000000,0.879771,0.956312,0.282646,0.210622,0.929173,0.174091,-0.544562,0.798102,-0.412992
Total number of GPT-3 calls made,0.365100,-0.067734,-0.218031,0.422258,0.879771,1.000000,0.931990,0.647249,0.192459,0.908722,0.395012,-0.610905,0.806284,-0.358701
Number of times GPT-3 suggestion is used,0.454037,-0.096746,-0.167396,0.451338,0.956312,0.931990,1.000000,0.326916,0.225656,0.970277,0.407231,-0.609296,0.847576,-0.404455
Number of times user rejected GPT-3 suggestion,-0.002934,0.026855,-0.216420,0.151763,0.282646,0.647249,0.326916,1.000000,0.027233,0.328759,0.173480,-0.311419,0.319716,-0.084651
Number of times GPT-3 suggestion is modified,0.051890,-0.056362,-0.100297,0.199498,0.210622,0.192459,0.225656,0.027233,1.000000,-0.016807,0.184228,-0.200982,0.269933,-0.053334
Number of times GPT-3 suggestion is used as is,0.453102,-0.085293,-0.146890,0.413665,0.929173,0.908722,0.970277,0.328759,-0.016807,1.000000,0.372191,-0.575415,0.802841,-0.401856


# View DataFrame

In [14]:
df.head()

,file_name,text,Total number of sentences,Number of sentences of initial prompt,Number of sentences completely authored by the user,Number of sentences completely authored by GPT-3,Number of sentences authored by GPT-3 and user,Total number of GPT-3 calls made,Number of times GPT-3 suggestion is used,Number of times user rejected GPT-3 suggestion,Number of times GPT-3 suggestion is modified,Number of times GPT-3 suggestion is used as is,GPT-3 : Total Sentences,User : Total Sentences,Amount of GTP-3 Usage,Type Token Ratio
0,8c11358444974bf0b5224183acd8149d.jsonl,What Stereotypical Characters Make You Cringe?...,15,4,6,0,5,5,5,0,3,2,0.00000,0.400000,0.333333,0.564706
1,c7dc5563ed07478f9284190b6085f4d3.jsonl,How Worried Should We Be About Screen Time Dur...,20,6,10,0,4,6,4,2,3,1,0.00000,0.500000,0.200000,0.613559
2,05a000131fc642f7bb20b62bb20a326e.jsonl,"All of the ""#1 Dad"" mugs in the world change t...",22,1,18,0,3,6,3,3,3,0,0.00000,0.818182,0.136364,0.548476
3,7834dec912b34643afb92b7c3648a3fe.jsonl,"When you die, you appear in a cinema with a nu...",32,2,18,1,11,12,12,0,0,12,0.03125,0.562500,0.375000,0.469974
4,105bf88bb4bc42688e06a54644e2989b.jsonl,"When you're 28, science discovers a drug that ...",37,3,24,0,10,13,10,3,0,10,0.00000,0.648649,0.270270,0.491135


# Export to CSV

In [15]:
df.to_csv("writing_session_stats.csv")

# Write Sentences to Text Files

In [16]:
# for file_name, text in zip(df["file_name"], df["text"]):
#     file_name = file_name.split('.')[0] # Extract only the name and not the 'jsonl' part
#     file_name = "taaco-input-texts/" + file_name + ".txt"
#     with open(file_name, "w") as f:
#         f.write(text)

# Read TAACO Metrics CSV

In [17]:
taaco_df = pd.read_csv("results.csv")
taaco_df.head()

,Filename,lemma_ttr,lemma_mattr,lexical_density_tokens,lexical_density_types,content_ttr,function_ttr,function_mattr,noun_ttr,verb_ttr,...,negative_logical,all_temporal,positive_intentional,all_positive,all_negative,all_connective,pronoun_density,pronoun_noun_ratio,repeated_content_lemmas,repeated_content_and_pronoun_lemmas
0,007769c9000e457eae8485221041802d.txt,0.378713,0.683549,0.475248,0.751634,0.598958,0.202830,0.405276,0.790698,0.500000,...,0.014851,0.014851,0.012376,0.061881,0.014851,0.066832,0.215347,2.023256,0.287129,0.502475
1,00bf170a815a42359f3aef35f5674ddc.txt,0.471850,0.803765,0.541555,0.750000,0.653465,0.280702,0.522951,0.650000,0.603448,...,0.002681,0.008043,0.000000,0.037534,0.008043,0.034853,0.056300,0.210000,0.262735,0.313673
2,00d39011efcb4533ab12076801f74f42.txt,0.326816,0.638641,0.444134,0.717949,0.528302,0.180905,0.402133,0.450000,0.587302,...,0.011173,0.016760,0.002793,0.053073,0.011173,0.067039,0.148045,0.883333,0.301676,0.444134
3,0139e814be15409dbab46c2d2d9ca07f.txt,0.422131,0.753075,0.508197,0.762136,0.633065,0.229167,0.473822,0.637168,0.571429,...,0.004098,0.010246,0.010246,0.079918,0.010246,0.067623,0.131148,0.566372,0.272541,0.401639
4,01650a401e614c38a04a904165a5784f.txt,0.503571,0.715758,0.478571,0.709220,0.746269,0.294521,0.433196,0.761905,0.666667,...,0.007143,0.010714,0.000000,0.053571,0.007143,0.046429,0.028571,0.126984,0.189286,0.196429


# Append TAACO Metrics to Original DataFrame 

In [18]:
taaco_df["file_name"] = taaco_df["Filename"].apply(lambda x: x.split(".")[0])
taaco_df.drop(["Filename"], inplace=True, axis=1)

In [19]:
new_df = df
new_df["file_name"] = new_df["file_name"].apply(lambda x: x.split(".")[0])

In [20]:
new_df = pd.merge(new_df, taaco_df, on="file_name")
new_df.head()

,file_name,text,Total number of sentences,Number of sentences of initial prompt,Number of sentences completely authored by the user,Number of sentences completely authored by GPT-3,Number of sentences authored by GPT-3 and user,Total number of GPT-3 calls made,Number of times GPT-3 suggestion is used,Number of times user rejected GPT-3 suggestion,...,negative_logical,all_temporal,positive_intentional,all_positive,all_negative,all_connective,pronoun_density,pronoun_noun_ratio,repeated_content_lemmas,repeated_content_and_pronoun_lemmas
0,8c11358444974bf0b5224183acd8149d,What Stereotypical Characters Make You Cringe?...,15,4,6,0,5,5,5,0,...,0.003937,0.011811,0.015748,0.078740,0.023622,0.094488,0.055118,0.245614,0.311024,0.358268
1,c7dc5563ed07478f9284190b6085f4d3,How Worried Should We Be About Screen Time Dur...,20,6,10,0,4,6,4,2,...,0.010204,0.017007,0.003401,0.074830,0.013605,0.088435,0.017007,0.058824,0.224490,0.224490
2,05a000131fc642f7bb20b62bb20a326e,"All of the ""#1 Dad"" mugs in the world change t...",22,1,18,0,3,6,3,3,...,0.002681,0.021448,0.002681,0.061662,0.010724,0.064343,0.045576,0.186813,0.222520,0.265416
3,7834dec912b34643afb92b7c3648a3fe,"When you die, you appear in a cinema with a nu...",32,2,18,1,11,12,12,0,...,0.007833,0.028721,0.005222,0.075718,0.007833,0.075718,0.065274,0.409836,0.263708,0.326371
4,105bf88bb4bc42688e06a54644e2989b,"When you're 28, science discovers a drug that ...",37,3,24,0,10,13,10,3,...,0.017730,0.019504,0.005319,0.072695,0.019504,0.072695,0.078014,0.478261,0.241135,0.313830


# Divide into High-usage and Low-usage Groups

In [21]:
print("Median of Amount of GPT-3 Usage:", np.median(new_df["Amount of GTP-3 Usage"]))

Median of Amount of GPT-3 Usage: 0.25


In [22]:
df_high = new_df[new_df["Amount of GTP-3 Usage"] > np.median(new_df["Amount of GTP-3 Usage"])]

In [23]:
df_low = new_df[new_df["Amount of GTP-3 Usage"] <= np.median(new_df["Amount of GTP-3 Usage"])]

# Print High-usage Group Metrics

In [24]:
for col in df_high.columns:
    if col in df.columns:
        continue
    print("Mean", col, ":", np.mean(df_high[col]))

Mean lemma_ttr : 0.39611582719532457
Mean lemma_mattr : 0.7182977377160918
Mean lexical_density_tokens : 0.4926949109058426
Mean lexical_density_types : 0.7296625080290305
Mean content_ttr : 0.586431661330968
Mean function_ttr : 0.2298191084586581
Mean function_mattr : 0.4598025822118426
Mean noun_ttr : 0.5718294673313705
Mean verb_ttr : 0.578616811517585
Mean adj_ttr : 0.7458869974028858
Mean adv_ttr : 0.650225766476837
Mean prp_ttr : 0.17768079241225962
Mean argument_ttr : 0.42991633349094843
Mean bigram_lemma_ttr : 0.8239844002802577
Mean trigram_lemma_ttr : 0.937303613016993
Mean adjacent_overlap_all_sent : 0.2209816823547981
Mean adjacent_overlap_all_sent_div_seg : 3.093252830333542
Mean adjacent_overlap_binary_all_sent : 0.8559451530767535
Mean adjacent_overlap_2_all_sent : 0.3255004566188468
Mean adjacent_overlap_2_all_sent_div_seg : 4.5227945074852505
Mean adjacent_overlap_binary_2_all_sent : 0.9328707260189346
Mean adjacent_overlap_cw_sent : 0.1177543106766741
Mean adjacent_ov

# Print Low-usage Group Metrics

In [25]:
for col in df_low.columns:
    if col in df.columns:
        continue
    print("Mean", col, ":", np.mean(df_low[col]))

Mean lemma_ttr : 0.40644688188353156
Mean lemma_mattr : 0.7358031241203737
Mean lexical_density_tokens : 0.4985475465362733
Mean lexical_density_types : 0.7250907969085123
Mean content_ttr : 0.5914766065546153
Mean function_ttr : 0.2428463920135673
Mean function_mattr : 0.4814854760680687
Mean noun_ttr : 0.582911587011923
Mean verb_ttr : 0.575548092035228
Mean adj_ttr : 0.7482037402667884
Mean adv_ttr : 0.6518250262489066
Mean prp_ttr : 0.17899325255223886
Mean argument_ttr : 0.4392265208983434
Mean bigram_lemma_ttr : 0.8510042890397267
Mean trigram_lemma_ttr : 0.955060558344478
Mean adjacent_overlap_all_sent : 0.20340912779507023
Mean adjacent_overlap_all_sent_div_seg : 2.791734198145305
Mean adjacent_overlap_binary_all_sent : 0.8361618813735054
Mean adjacent_overlap_2_all_sent : 0.30347431711387773
Mean adjacent_overlap_2_all_sent_div_seg : 4.130514578567157
Mean adjacent_overlap_binary_2_all_sent : 0.9179715343113309
Mean adjacent_overlap_cw_sent : 0.10753751800784793
Mean adjacent_

# Collect Group Metrics into New DataFrame

In [26]:
group_metrics = pd.DataFrame()

In [27]:
metrics = []
low_group_val = []
high_group_val = []

In [28]:
for col in new_df.columns:
    if col in df.columns:
        continue
    metrics.append(col)
    low_group_val.append(np.mean(df_low[col]))
    high_group_val.append(np.mean(df_high[col]))
group_metrics["Metric"] = metrics
group_metrics["Low Group Value (Mean)"] = low_group_val
group_metrics["High Group Value (Mean)"] = high_group_val

group_metrics.to_csv("Group Metrics.csv")

In [29]:
group_metrics

,Metric,Low Group Value (Mean),High Group Value (Mean)
0,lemma_ttr,0.406447,0.396116
1,lemma_mattr,0.735803,0.718298
2,lexical_density_tokens,0.498548,0.492695
3,lexical_density_types,0.725091,0.729663
4,content_ttr,0.591477,0.586432
...,...,...,...
151,all_connective,0.068715,0.068288
152,pronoun_density,0.068136,0.069544
153,pronoun_noun_ratio,0.353257,0.357026
154,repeated_content_lemmas,0.288680,0.286841


# Save All Metrics for each Writing Session as CSV

In [30]:
new_df.to_csv("all metrics.csv")

# Generate Mean, Median and Standard Deviation for all Metrics

In [54]:
all_metrics_stats_df = pd.DataFrame()

metrics = []
mean = []
median = []
std = []
minimum = []
maximum = []

for col in new_df.columns:
    if col in ["file_name", "text"]: continue
    metrics.append(col)
    mean.append(np.mean(new_df[col]))
    median.append(np.median(new_df[col]))
    std.append(np.std(new_df[col]))
    minimum.append(np.min(new_df[col]))
    maximum.append(np.max(new_df[col]))

all_metrics_stats_df["Metrics"] = metrics
all_metrics_stats_df["Mean"] = mean
all_metrics_stats_df["Median"] = median
all_metrics_stats_df["Standard Deviation"] = std
all_metrics_stats_df["Minimum"] = minimum
all_metrics_stats_df["Maximum"] = maximum

all_metrics_stats_df.to_csv("All Metrics Stats.csv")
all_metrics_stats_df

,Metrics,Mean,Median,Standard Deviation,Minimum,Maximum
0,Total number of sentences,28.962656,27.000000,10.388910,11.000000,78.000000
1,Number of sentences of initial prompt,4.421162,4.000000,2.390986,0.000000,9.000000
2,Number of sentences completely authored by the...,16.242739,15.000000,9.535179,0.000000,64.000000
3,Number of sentences completely authored by GPT-3,0.685339,0.000000,1.886442,0.000000,22.000000
4,Number of sentences authored by GPT-3 and user,7.613416,6.000000,5.953073,0.000000,42.000000
...,...,...,...,...,...,...
165,all_connective,0.068503,0.067545,0.017166,0.017182,0.131078
166,pronoun_density,0.068835,0.062363,0.032258,0.008086,0.224000
167,pronoun_noun_ratio,0.355129,0.300000,0.230119,0.031250,2.023256
168,repeated_content_lemmas,0.287767,0.290206,0.045190,0.138996,0.436620


# T-Test

In [31]:
from scipy.stats import ttest_ind

In [48]:
t_test_df = pd.DataFrame()

t_stat = []
p_val = []
metrics = []
low_group_val = []
high_group_val = []

ALPHA = 0.025

for col in new_df.columns:
    if col in df.columns:
        continue
    metrics.append(col)
    t_stat_result, p_val_result = ttest_ind(df_low[col], df_high[col], equal_var=True)
    t_stat.append(t_stat_result)
    p_val.append(p_val_result)
    low_group_val.append(np.mean(df_low[col]))
    high_group_val.append(np.mean(df_high[col]))
    
t_test_df["Metric"] = metrics
t_test_df["Low Group Value (Mean)"] = low_group_val
t_test_df["High Group Value (Mean)"] = high_group_val
t_test_df["T-Statistic"] = t_stat
t_test_df["P-Value"] = p_val
t_test_df["Verdict (alpha=" + str(ALPHA) + ")"] = t_test_df["P-Value"].apply(lambda x:
    "Means of both groups are same" if x > ALPHA else "Means of both groups are different")

t_test_df.to_csv("T Test Results.csv")
t_test_df

,Metric,Low Group Value (Mean),High Group Value (Mean),T-Statistic,P-Value,Verdict (alpha=0.025)
0,lemma_ttr,0.406447,0.396116,3.592373,3.386763e-04,Means of both groups are different
1,lemma_mattr,0.735803,0.718298,8.631911,1.566870e-17,Means of both groups are different
2,lexical_density_tokens,0.498548,0.492695,2.909302,3.677817e-03,Means of both groups are different
3,lexical_density_types,0.725091,0.729663,-2.320651,2.044443e-02,Means of both groups are different
4,content_ttr,0.591477,0.586432,1.303750,1.925266e-01,Means of both groups are same
...,...,...,...,...,...,...
151,all_connective,0.068715,0.068288,0.472978,6.363001e-01,Means of both groups are same
152,pronoun_density,0.068136,0.069544,-0.829439,4.069933e-01,Means of both groups are same
153,pronoun_noun_ratio,0.353257,0.357026,-0.311179,7.557094e-01,Means of both groups are same
154,repeated_content_lemmas,0.288680,0.286841,0.773088,4.395967e-01,Means of both groups are same
